# SMO implementation
Attempting to implement the SMO (https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr-98-14.pdf). 

Also consider https://www.csie.ntu.edu.tw/~cjlin/papers/bottou_lin.pdf

First step, we wil need to be able to evaluate the SVM on
a single instance. 

I initially explored hmatrix, but then I realised that I don't need matrix inversion for this algorithm, so using Repa would be better
`--import Numeric.LinearAlgebra`


In [1]:
{- LANGUAGE XTypeOperators -}
import  Data.Array.Repa
import qualified Data.Array.Repa as R


In [2]:
import Data.Array.Repa.Repr.Vector

In [3]:
import Data.Array.Repa.Repr.Vector                   as RV

In [4]:
:i RV.fromListVector

In [5]:
type BaseVector = Array U DIM1 Double
type BaseScalar = Array U Z Double

type Sample = BaseVector
type Weights = BaseVector
type Threshold = BaseScalar

-- There is probably an opportunity to build a type-class around the 
-- rules for composing kernel functions. 
type Kernel = BaseVector -> BaseVector -> BaseScalar

data ClassLabel = Class1 | Class2 deriving (Show, Eq)


chooseClass :: Double -> ClassLabel
chooseClass res = if res >= 0 then Class1 else Class2
             
dot :: Kernel
dot a b =  foldS (+) 0 ( a *^ b) 

svm :: Kernel -> Weights -> Threshold -> Sample -> ClassLabel
svm k w b x = let
                res = (w `k` x) -^ b
              in 
                chooseClass  $ res ! Z
                
linearSvm :: Weights -> Threshold -> Sample -> ClassLabel
linearSvm = svm dot

In [6]:
x_inputs = [1..10] :: [Double]
w_inputs = [11..20] :: [Double]
x = fromListUnboxed (Z :. (10::Int))  x_inputs
w = fromListUnboxed (Z :. (10::Int))  w_inputs

thresh = fromListUnboxed Z ([2] :: [Double])

svm dot w thresh x 

Class1

In [7]:
largethresh = fromListUnboxed Z ([1000] :: [Double])

svm dot w largethresh x 

Class2

# Build Helper Functions 
Now that we can evaluate the SVM we need to work towards building the simple functions described in the paper. 

In [8]:
-- equation 13
lowerAlpha :: (Num val, Ord val) => val -> Bool -> val -> val -> val
lowerAlpha c True a1 a2 = max 0 (a1 - a2)
lowerAlpha c False a1 a2 = max 0 (a1 - a2 - c)

-- equation 14
upperAlpha :: (Num val, Ord val) => val -> Bool -> val -> val -> val
upperAlpha c True a1 a2 = min c (c + a1 - a2)
upperAlpha c False a1 a2 = min c (a1 - a2)
                                

In [9]:
-- Make a scalar multiplication operator
(.*) :: Double -> BaseScalar -> BaseScalar
(.*) val vec = R.computeUnboxedS $ R.map (\x -> val*x) vec

-- Make Vector into a scalar
squish :: BaseVector -> BaseScalar
squish v = computeS $ backpermute (Z) (undefined) v 

unsquish :: BaseScalar -> BaseVector
unsquish v = computeS $ backpermute (undefined) (undefined) v

-- equation 15, 2nd Derivative
eta :: Kernel -> Sample -> Sample -> BaseScalar
eta k x1 x2 =  R.computeUnboxedS $ (x1 `k` x1) +^ (x2 `k` x2) -^ (2 .* (x1 `k` x2) )

In [10]:
eta dot x w

AUnboxed Z [1000.0]

In [48]:
wrapScalar :: Double -> BaseScalar
wrapScalar s = fromListUnboxed Z ([s] :: [Double])


classToDbl ::  ClassLabel -> Double
classToDbl Class1 = 1
classToDbl Class2 = -1

classToScalar :: ClassLabel -> BaseScalar
classToScalar a = wrapScalar $ classToDbl a


In [46]:
classError :: Num a => ClassLabel -> ClassLabel -> Double
classError label1 label2 = (classToDbl label1) - ( classToDbl label2)

In [50]:
-- equation 16, minimum along contstraint direction
alpha2New :: Double      -- Current Alpha Value
             -> Double   -- Result of the graidient calculation
             -> ClassLabel   -- TrueLabel of point1
             -> ClassLabel   -- TrueLabel of point2
             -> ClassLabel   -- Predicted Label of point1
             -> ClassLabel   -- Predicted Label of point2 
             -> Double   -- New alpha
alpha2New a e y1 y2 s1 s2 = 
    let 
        y2' = classToDbl y2     
        e1 = classError y1 s1   
        e2 = classError y2 s2   
    in
        a + y2' * (e1 - e2) / e -- Implementing eq 16
        

In [51]:
-- Equation 17, 
alphaNewClipped :: (Num a, Ord a) => a -- alpha2New 
                   -> a                -- H
                   -> a                -- L
                   -> a
alphaNewClipped a h l 
    | a >= h = h -- will fail typechecker might need to cast or something...
    | a <= l = l
    | otherwise = a

In [53]:
-- Equation 18

alpha1New :: Double   -- Alpha1
                    -> ClassLabel -- True Label class1
                    -> ClassLabel -- True Label class2
                    -> Double -- Alpha2
                    -> Double -- Alpha2Clipped
                    -> Double
alpha1New a y1 y2 a2 a2clip =
    let 
        y1' = classToDbl y1
        y2' = classToDbl y2
        s = y1' * y2' 
    in 
        a + s * (a2 - a2clip)